#**Feature selection using reinforcement learning and Bees optimization algorithm**

<a href="https://colab.research.google.com/github/Neofly4023/bso-fs/blob/master/bso_fs_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Feature selection playing a major role in ML tasks. In this project, we’re proposing a reinforcement learning and Bees optimization algorithm. We had tune the parameter also for more accurate classification process




Created By: 
*   Areej Turky AlOtaibi 
*   Raghad Adel AlShabana
*   Shoroog Saad AlArifi
*   Slima Mohammad Bnous

##Installing requirement

In [15]:
!pip install pip==21.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
!pip install xlsxwriter pandas numpy sklearn matplotlib hyperopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, glob, random, copy, sys, time, re, operator, xlsxwriter
from collections import defaultdict
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold, StratifiedKFold, \
ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from oauth2client.client import GoogleCredentials
from google.colab import drive
from prettytable import PrettyTable
from sklearn.metrics import accuracy_score
from hyperopt import hp, fmin, tpe, Trials
import matplotlib.pyplot as plt


In [18]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Bee class
This class is used to implement the bee and the local search

In [19]:
class Bee :
    def __init__(self,id,problem,locIterations):
        self.id=id
        self.data=problem
        self.solution=[]
        self.fitness= 0.0
        self.reward = 0.0
        self.locIterations=locIterations
        self.action = []
    
    def localSearch(self):
        best=self.fitness
        #done=False
        lista=[j for j, n in enumerate(self.solution) if n == 1]
        indice =lista[0]
        
        for itr in range(self.locIterations):
            while(True):
                pos=-1
                oldFitness=self.fitness
                for i in range(len(self.solution)):
                    
                    if ((len(lista)==1) and (indice==i) and (i < self.data.nb_attribs-1)):
                        i+=1
                    self.solution[i]= (self.solution[i] + 1) % 2
                    
                    quality = self.data.evaluate(self.solution)
                    if (quality >best):
                        pos = i
                        best=quality
                    self.solution[i]= (self.solution[i]+1) % 2
                    self.fitness = oldFitness 
                if (pos != -1):
                    self.solution[pos]= (self.solution[pos]+1)%2
                    self.fitness = best
                else:
                    break
            for i in range(len(self.solution)):
                oldFitness=self.fitness
                if ((len(lista)==1) and (indice==i) and (i < self.data.nb_attribs-1)):
                    i+=1
                self.solution[i]= (self.solution[i] + 1) % 2
                quality = self.data.evaluate(self.solution)
                if (quality<best):
                    self.solution[i]= (self.solution[i] + 1) % 2
                    self.fitness = oldFitness


    def ql_localSearch(self):
        
        iterations = int(self.locIterations/4) if self.locIterations >= 4 else 1
        for itr in range(iterations):
       
            state = self.solution.copy()

            next_state, action = self.data.ql.step(self.data,state)
            acc_state = self.data.evaluate(state)
            acc_new_state = self.data.evaluate(next_state)

            if (acc_state < acc_new_state):
                self.reward = acc_new_state
            elif (acc_state > acc_new_state):
                self.reward = acc_new_state - acc_state
            else :
                if (self.data.nbrUn(state) > self.data.nbrUn(next_state) ):
                    self.reward = 0.5 * acc_new_state
                else :
                    self.reward = -0.5 * acc_new_state

            self.fitness = self.data.ql.get_q_value(self.data,state,action)
            self.data.ql.learn(self.data,state,action,self.reward,next_state)
            self.solution = next_state.copy()
        
       
    def setSolution(self,solution):
        self.solution=solution.copy()
        self.fitness=self.data.evaluate(solution)
    
    def Rand(self,num): 
        res = [] 
        res = np.random.choice([0,1],size=(num,),p=[2./10,8./10]).tolist()
  
        return res


##Swarm class 
This class is used to initilize the bees and the search area

In [20]:
class Swarm :
    def __init__(self,problem,flip,maxChance,nbrBees,maxIterations,locIterations):
        self.data=problem
        self.flip=flip
        self.maxChance=maxChance
        self.nbChance=maxChance
        self.nbrBees=nbrBees
        self.maxIterations=maxIterations
        self.locIterations=locIterations
        self.beeList=[]
        self.refSolution = Bee(-1,self.data,self.locIterations)
        self.refSolution.setSolution(self.refSolution.Rand(self.data.nb_attribs))
        self.bestSolution = self.refSolution
        self.tabou=[]

    def searchArea(self):    
        i=0
        h=0
        
        self.beeList=[]
        while((i<self.nbrBees) and (i < self.flip) ) :
            
            solution=self.refSolution.solution.copy()
            k=0
            while((self.flip*k+h) < len(solution)):
                solution[self.flip*k +h] = ((solution[self.flip*k+h]+1) % 2)
                k+=1
            newBee=Bee(i,self.data,self.locIterations)
            newBee.solution = solution.copy()
            self.beeList.append(newBee)
            
            i+=1
            h=h+1
        h=0
        
        while((i<self.nbrBees) and (i< 2*self.flip )):

            solution=self.refSolution.solution.copy()
            k=0
            while((k<int(len(solution)/self.flip)) and (self.flip*k+h < len(solution))):
                solution[int(self.data.nb_attribs/self.flip)*h+k] = ((solution[int(self.data.nb_attribs/self.flip)*h+k]+1)%2)
                k+=1
            newBee=Bee(i,self.data,self.locIterations)
            newBee.solution = solution.copy()
            self.beeList.append(newBee)
            
            i+=1
            h=h+1
        while (i<self.nbrBees):
            solution= self.refSolution.solution.copy()
            indice = random.randint(0,len(solution)-1)
            solution[indice]=((solution[indice]+1) % 2)
            newBee=Bee(i,self.data,self.locIterations)
            newBee.solution = solution.copy()
            self.beeList.append(newBee)
            i+=1
        for bee in (self.beeList):
            lista=[j for j, n in enumerate(bee.solution) if n == 1]
            if (len(lista)== 0):
                bee.setSolution(bee.Rand(self.data.nb_attribs))
                
    def selectRefSol(self,typeOfAlgo):
      typeOfAlgo = typeOfAlgo
      if (typeOfAlgo == 0):
        self.beeList.sort(key=lambda Bee: Bee.fitness, reverse=True)
        bestQuality=self.beeList[0].fitness
        if(bestQuality>self.bestSolution.fitness):
            self.bestSolution=self.beeList[0]
            self.nbChance=self.maxChance
            return self.bestSolution
        else:
            if(  (len(self.tabou)!=0) and  bestQuality > (self.tabou[len(self.tabou)-1].fitness)  ):
                self.nbChance=self.maxChance
                return self.bestBeeQuality(typeOfAlgo)
            else:
                self.nbChance-=1
                if(self.nbChance > 0): 
                    return self.bestBeeQuality(typeOfAlgo)
                else :
                    return self.bestBeeDiversity()
      
      elif (typeOfAlgo == 1):
        self.beeList.sort(key=lambda Bee: Bee.reward, reverse=True)
        bestQuality=self.beeList[0].reward
        if(bestQuality>self.bestSolution.reward):
            self.bestSolution=self.beeList[0]
            self.nbChance=self.maxChance
            return self.bestSolution
        else:
            if(  (len(self.tabou)!=0) and  bestQuality > (self.tabou[len(self.tabou)-1].reward)  ):
                self.nbChance=self.maxChance
                return self.bestBeeQuality(typeOfAlgo)
            else:
                self.nbChance-=1
                if(self.nbChance > 0): 
                    return self.bestBeeQuality(typeOfAlgo)
                else :
                    return self.bestBeeDiversity()                  

    def distanceTabou(self,bee):
        distanceMin=self.data.nb_attribs
        for i in range(len(self.tabou)):
            cpt=0
            for j in range(self.data.nb_attribs):
                if (bee.solution[j] != self.tabou[i].solution[j]) :
                      cpt +=1
            if (cpt<=1) :
                return 0
            if (cpt < distanceMin) :
                distanceMin=cpt
        return distanceMin
    
    def bestBeeQuality(self,typeOfAlgo):
        distance = 0
        i=0
        pos=-1
        while(i<self.nbrBees):
            if (typeOfAlgo == 0):
              max_val=self.beeList[i].fitness
            if (typeOfAlgo == 1):
              max_val=self.beeList[i].reward  

            nbUn=self.data.nbrUn(self.beeList[i].solution)
            while((i<self.nbrBees) and (self.data.evaluate(self.beeList[i].solution) == max_val)):
                distanceTemp=self.distanceTabou(self.beeList[i])
                nbUnTemp = self.data.nbrUn(self.beeList[i].solution)
                if(distanceTemp > distance) or ((distanceTemp == distance) and (nbUnTemp < nbUn)):
                    if((distanceTemp==distance) and (nbUnTemp<nbUn)):
                        print("We pick the solution with less features")
                    nbUn=nbUnTemp
                    distance=distanceTemp
                    pos=i
                i+=1
            if(pos!=-1) :
                return self.beeList[pos]
        bee= Bee(-1,self.data,self.locIterations)
        bee.setSolution(bee.Rand(self.data.nb_attribs))
        return bee
            
    def bestBeeDiversity(self):
        max_val=0
        for i in range(len(self.beeList)):
            if (self.distanceTabou(self.beeList[i])> max_val) :
                max_val = self.distanceTabou(self.beeList[i])
        if (max_val==0):
            bee= Bee(-1,self.data,self.locIterations)
            bee.setSolution(bee.Rand(self.data.nb_attribs))
            return bee
        i=0
        while(i<len(self.beeList) and self.distanceTabou(self.beeList[i])!= max_val) :
            i+=1
        return self.beeList[i]
    
    def bso(self,typeOfAlgo):
        i=0
        while(i<self.maxIterations):
            self.tabou.append(self.refSolution)
        
            
            self.searchArea()
            
            for j in range(self.nbrBees):
              if (typeOfAlgo == 0):
                self.beeList[j].localSearch()
              elif (typeOfAlgo == 1):
                for episode in range(self.locIterations):
                  self.beeList[j].ql_localSearch()
            self.refSolution = self.selectRefSol(typeOfAlgo)
            i+=1
        print("[BSO parameters used]\n")
        print("Type of algo : {0}".format(typeOfAlgo))
        print("Flip : {0}".format(self.flip))
        print("MaxChance : {0}".format(self.maxChance))
        print("Nbr of Bees : {0}".format(self.nbrBees))
        print("Nbr of Max Iterations : {0}".format(self.maxIterations))
        print("Nbr of Loc Iterations : {0}\n".format(self.locIterations))
        print("Best solution found : ",self.bestSolution.solution)
        print("Number of features used : {0}".format(self.data.nbrUn(self.bestSolution.solution)))
        print("Accuracy : {0:.2f} ".format(self.bestSolution.fitness*100))
        #print("Return (Q-value) : ",self.bestSolution.fitness)
        return self.bestSolution.fitness*100, self.data.nbrUn(self.bestSolution.solution)

    
    def str_sol(self,mlist):
        result = ''
        for element in mlist:
            result += str(element)
        return result


## Reinforcement class
This class is used to implement the Q-learning algorithm and update the q_table

In [21]:
class QLearning:
    def __init__(self,nb_atts,actions):
        self.actions = actions
        self.alpha = 0.1 
        self.gamma = 0.9
        self.epsilon = 0.1
        self.q_table = [ {} for i in range(nb_atts) ] 

    def get_max_value(self,data,state,actions_vals):
        max_val = 0.0
        arg_max = 0
        for i in actions_vals:
            if self.get_q_value(data,state,i) >= max_val:
                max_val = self.get_q_value(data,state,i)
                arg_max = i
        if max_val == 0:
            arg_max = np.random.choice(actions_vals)
        return max_val,arg_max


    def get_q_value(self,data,state,action):
        if not self.str_sol(state) in self.q_table[self.nbrUn(state)]:
            self.q_table[self.nbrUn(state)][self.str_sol(state)] = {}

        if not str(action) in self.q_table[self.nbrUn(state)][self.str_sol(state)]:
            self.q_table[self.nbrUn(state)][self.str_sol(state)][str(action)] = data.evaluate(self.get_next_state(state,action))
            self.q_table[self.nbrUn(state)][self.str_sol(state)][str(action)] = 0
            
        return self.q_table[self.nbrUn(state)][self.str_sol(state)][str(action)]

    def set_q_value(self,state,action,val):
        self.q_table[self.nbrUn(state)][self.str_sol(state)][str(action)] = val


    def step(self,data,state):
        if np.random.uniform() > self.epsilon :
            action_values = self.actions
            argmax_actions=[] 
            for ac in action_values :

                ac_state_q_val = self.get_q_value(data,state,ac)
                if ( ac_state_q_val >= self.get_max_value(data,state,action_values)[0] ):
                    argmax_actions.append(ac)

            if len(argmax_actions) != 0:
              next_action = np.random.choice(action_values) 
            else:
              next_action = np.random.choice(action_values) 
            next_state = self.get_next_state(state,next_action)
            
        else :
            next_action = np.random.choice(self.actions)
            next_state = self.get_next_state(state,next_action)


        if self.epsilon > 0 :
            self.epsilon -= 0.0001 
        if self.epsilon < 0 :
            self.epsilon = 0

        return next_state, next_action #, reward


    def get_next_state(self,state,action):
        next_state = state.copy()
        next_state[action] = (next_state[action]+1) % 2
        if (self.nbrUn(next_state) != 0):
          return next_state
        else:
          return state
    
    def learn(self,data,current_state,current_action,reward,next_state):
        
        next_action = self.step(data,next_state)[1] # step returns 3 values : next_state, next_action, and the reward
        new_q = reward + self.gamma * self.get_q_value(data,next_state,next_action)  #[0] is to pick q-value instead of [1] which is the accuracy of the new state 
        self.set_q_value(current_state,current_action,(1 - self.alpha)*self.get_q_value(data,current_state,current_action) + self.alpha*new_q)  

    def str_sol(self,mlist):
        result = ''
        for element in mlist:
            result += str(element)
        return result

    def nbrUn(self,solution):
        return len([i for i, n in enumerate(solution) if n == 1])

    def set_parameters(self, alpha, gamma, eps):
      self.alpha= alpha
      self.gamma= gamma
      self.epsilon=eps
  


##Feature Selection class
This class is used to initilize the features and labels from the dataset

In [22]:
class FsProblem :
    def __init__(self,data,qlearn):
        self.data=data
        self.nb_attribs= len(self.data.columns)-1 
        self.outPuts=self.data.iloc[:,self.nb_attribs]
        self.ql = qlearn
        self.nb_actions = len(self.ql.actions)
        self.classifier = KNeighborsClassifier(n_neighbors=1)
        self.x= self.data.iloc[:,0:self.nb_attribs]     #Initilize the features from the dataset
        self.y= self.outPuts                          #Initilize the labels from the dataset
        #After tuning the K, reinitilize clf
        ######In case of Hyperopt########
        self.k= self.tuning_hyperOp()
        ########In case of cross_valis
        #self.k= self.tuning_CV()########
        self.classifier= KNeighborsClassifier(n_neighbors=self.k)
        
    def evaluate2(self,solution):
        list=[i for i, n in enumerate(solution) if n == 1]
        if (len(list) == 0):
            return 0
         
        df = self.data.iloc[:,list]
        array=df.values
        nb_attribs =len(array[0])
        X = array[:,0:nb_attribs]
        Y = self.outPuts
        train_X, test_X, train_y, test_y = train_test_split(X, Y, 
                                                    random_state=0,
                                                    test_size=0.1
                                                    )
        classifier = KNeighborsClassifier(n_neighbors=1)
        classifier.fit(train_X,train_y)
        predict= classifier.predict(test_X) 
        return metrics.accuracy_score(predict,test_y)
    
    def evaluate(self,solution):
        list=[i for i, n in enumerate(solution) if n == 1]
        if (len(list)== 0):
            return 0
        df = self.data.iloc[:,list]        
        array=df.values
        nbrAttributs =len(array[0])
        X = array[:,0:nbrAttributs]
        Y = self.outPuts
        cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
        results = cross_val_score(self.classifier, X, Y, cv=cv,scoring='accuracy')
        return results.mean()
        
    def nbrUn(self,solution):
        return len([i for i, n in enumerate(solution) if n == 1])
      #The hyper optimization method for tuning
    def tuning_hyperOp(self):
      # Parameter bounds
      x_train, x_test, y_train, y_test = train_test_split(self.x, self.y, test_size=0.2, random_state=0)
      space = {
      'k': hp.choice('k', range(1, 11))  # Assuming k ranges from 1 to 10
      }
      #Objective function that train the classifier
      def objective(params):
          k = params['k']
          # Create the k-nearest neighbors classifier
          clf = KNeighborsClassifier(n_neighbors=k)
          # Fit the classifier on the training data
          clf.fit(x_train, y_train)
          # Predict the labels for the test data
          y_pred = clf.predict(x_test)
          # Compute the accuracy score
          accuracy = accuracy_score(y_test, y_pred)
          return -accuracy

      trials = Trials()
      best = fmin(objective, space, algo=tpe.suggest, max_evals=10, trials=trials)

      optimal_k = best['k']
      print("Optimal_k: ", optimal_k)
    
      return optimal_k
  #The cross validation method for tuning
    def tuning_CV(self):
        parameterlist_k=[]
        k=[3,5,10]
        max_k=0

        for i in k:
            scores = cross_validate(self.classifier, self.x, self.y , cv=3, scoring='accuracy')
            parameterlist_k.append([i, scores])

        print(parameterlist_k)
        #Print the best parameter
        print("Best parameter: ", max(parameterlist_k, key=lambda x: x[1]['test_score'].mean()))
        #The k with the best score
        max_k=max(parameterlist_k, key=lambda x: x[1]['test_score'].mean())[0]
        #print the max k
        print("Max k: ", max_k)
        return max_k  
 

##Feature selection data
This class is used to train the classifier and evaluate the accuracy of the solution

In [31]:
class FSData:

    def __init__(self,location,nbr_exec):
        
        self.location = location
        self.nb_exec = nbr_exec
        self.dataset_name = re.search('[A-Za-z\-]*.csv',self.location)[0].split('.')[0]
        self.df = pd.read_csv(self.location,header=None)
        self.ql = QLearning(len(self.df.columns),self.attributs_to_flip(len(self.df.columns)-1))
        self.fsd = FsProblem(self.df,self.ql)
        self.results = []
        self.classifier_name = str(type(self.fsd.classifier)).strip('< > \' class ').split('.')[3]

        print("[START] Dataset" + self.dataset_name + "description \n")
        print("Shape : " + str(self.df.shape) + "\n")
        print(self.df.describe())
        print("\n[END] Dataset" + self.dataset_name + "description\n")
        print("[START] Ressources specifications\n")
        print("[END] Ressources specifications\n")
    
    def attributs_to_flip(self,nb_att):
        return list(range(nb_att))
    
    def run(self,typeOfAlgo,flip,maxChance,nbrBees,maxIterations,locIterations):

        t_init = time.time()
        
        for itr in range(1,self.nb_exec+1):
          print ("Execution N°{0}".format(str(itr)))
          self.ql = QLearning(len(self.df.columns),self.attributs_to_flip(len(self.df.columns)-1))
          self.fsd = FsProblem(self.df,self.ql)
          swarm = Swarm(self.fsd,flip,maxChance,nbrBees,maxIterations,locIterations)
          t1 = time.time()
          best = swarm.bso(typeOfAlgo)
          t2 = time.time()
          print("Time elapsed for execution N°{0} : {1:.2f} s\n".format(itr,t2-t1))
          
        t_end = time.time()
        print ("Total execution time for dataset {0} is {1:.2f} s".format(self.dataset_name,t_end-t_init))


##Main 
Has a parameter, call of the training and the output

In [32]:
# Prepare the dataset
data_loc_path = "/content/drive/MyDrive/Colab Notebooks/Optimization/Iris.csv"

#Params init
typeOfAlgo = 0
nbr_exec = 1
flip = 5
maxChance = 3
nbrBees = 10
maxIterations = 2
locIterations = 2

instance = FSData(data_loc_path,nbr_exec)
instance.run(typeOfAlgo,flip,maxChance,nbrBees,maxIterations,locIterations)


100%|██████████| 10/10 [00:00<00:00, 51.58trial/s, best loss: -1.0]
Optimal_k:  5
[START] DatasetIrisdescription 

Shape : (150, 5)

                0           1           2           3
count  150.000000  150.000000  150.000000  150.000000
mean     5.843333    3.054000    3.758667    1.198667
std      0.828066    0.433594    1.764420    0.763161
min      4.300000    2.000000    1.000000    0.100000
25%      5.100000    2.800000    1.600000    0.300000
50%      5.800000    3.000000    4.350000    1.300000
75%      6.400000    3.300000    5.100000    1.800000
max      7.900000    4.400000    6.900000    2.500000

[END] DatasetIrisdescription

[START] Ressources specifications

[END] Ressources specifications

Execution N°1
100%|██████████| 10/10 [00:00<00:00, 54.31trial/s, best loss: -1.0]
Optimal_k:  6
[BSO parameters used]

Type of algo : 0
Flip : 5
MaxChance : 3
Nbr of Bees : 10
Nbr of Max Iterations : 2
Nbr of Loc Iterations : 2

Best solution found :  [1, 0, 1, 1]
Number of feature